In [2]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [11]:
source = dict({'user_1': 150, 
               'user_2': 150})
print("source:", source)

# maximum amount of requests that can come through a node. Unlimited.
# Currently, they are set to be sum of total supply. It means no throughput constraint effectively.
# It could be useful later to limit the cross cluster routing.
node = dict({'A_start_1': 200, 
            'A_end_1': 200,
            
            'A_start_2': 200,
            'A_end_2': 200,
            
            'B_start_1': 200,
            'B_end_1': 200,
            
            'B_start_2': 200,
            'B_end_2': 200
            })
print("node:", node)

total_num_request = 0
for key, value in source.items():
    total_num_request += value
destination = dict({'dst': total_num_request}) # [user_1 supply] + [user_2 supply]
print("destination:", destination)

network_latency = dict({"region":80.0,
                        "rack":1.0})

# They are variable
service_time = dict({"A":30.0,
                     "B":10.0})

# topology: A -> B -> C
arcs, cost = gp.multidict({
    # user_1: end user connected to cluster 1
    # user_2: end user connected to cluster 2
    # ('user_1', 'A_start_1'): network_latency["rack"],
    # ('user_2', 'A_start_2'): network_latency["rack"],
    ('user_1', 'A_start_1'): 0,
    ('user_2', 'A_start_2'): 0,
    ('user_1', 'A_start_2'): network_latency["region"],
    ('user_2', 'A_start_1'): network_latency["region"],
    
    # service time of A, it should be function of something.
    ('A_start_1', 'A_end_1'): service_time["A"],
    ('A_start_2', 'A_end_2'): service_time["A"],
    
    # network latency between A and B
    ('A_end_1', 'B_start_1'): network_latency["rack"],
    ('A_end_2', 'B_start_2'): network_latency["rack"],
    ('A_end_1', 'B_start_2'): network_latency["region"],
    ('A_end_2', 'B_start_1'): network_latency["region"],
    
    # service time of B, it should be function of something.
    ('B_start_1', 'B_end_1'): service_time["B"],
    ('B_start_2', 'B_end_2'): service_time["B"],
    
    ('B_end_1', 'dst'): 0.0,
    ('B_end_2', 'dst'): 0.0,
})

# # print(type(arcs))
# print("arcs: ", arcs)
# # print(dict(arcs).keys())
# # print(type(cost))
# print("cost: ", cost)

model = gp.Model('RequestRouting')
flow = model.addVars(arcs, obj=cost, name="flow")

# Constraint 1: source
src_keys = source.keys()
src_flow = model.addConstrs((gp.quicksum(flow.select(src, '*')) == source[src] for src in src_keys), name="source")

# Constraint 2: destination
dest_keys = destination.keys()
dst_flow = model.addConstrs((gp.quicksum(flow.select('*', dst)) == destination[dst] for dst in dest_keys), name="destination")

# Constraint 3: flow conservation
node_key = node.keys()
node_flow = model.addConstrs((gp.quicksum(flow.select(n_, '*')) == gp.quicksum(flow.select('*', n_)) for n_ in node_key), name="flow_conservation")

# Constraint 4: max throughput of service
node_key = node.keys()
throughput = model.addConstrs((gp.quicksum(flow.select('*', n_)) <= node[n_] for n_ in node_key), name="service_capacity")

cost

source: {'user_1': 150, 'user_2': 150}
node: {'A_start_1': 200, 'A_end_1': 200, 'A_start_2': 200, 'A_end_2': 200, 'B_start_1': 200, 'B_end_1': 200, 'B_start_2': 200, 'B_end_2': 200}
destination: {'dst': 300}


{('user_1', 'A_start_1'): 0,
 ('user_2', 'A_start_2'): 0,
 ('user_1', 'A_start_2'): 80.0,
 ('user_2', 'A_start_1'): 80.0,
 ('A_start_1', 'A_end_1'): 30.0,
 ('A_start_2', 'A_end_2'): 30.0,
 ('A_end_1', 'B_start_1'): 1.0,
 ('A_end_2', 'B_start_2'): 1.0,
 ('A_end_1', 'B_start_2'): 80.0,
 ('A_end_2', 'B_start_1'): 80.0,
 ('B_start_1', 'B_end_1'): 10.0,
 ('B_start_2', 'B_end_2'): 10.0,
 ('B_end_1', 'dst'): 0.0,
 ('B_end_2', 'dst'): 0.0}

In [12]:
model.optimize()
if model.Status == GRB.INFEASIBLE:
    print("######################")
    print("### INFEASIBLE MODEL!")
    model.computeIIS()
    model.write("model.ilp")
    # print('\nThe following constraints and variables are in the IIS:')
    # for c in model.getConstrs():
    #     if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')
    # for v in model.getVars():
    #     if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
    #     if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')
    
# if model.Status == GRB.OPTIMAL:
#     solution = model.getAttr('X', flow)
#     for i, j in arcs:
#         if solution[i, j] > 0:
#             print('%s -> %s: %g' % (i, j, solution[i, j]))

# Variable info
varInfo = [(v.varName, v.LB, v.UB) for v in model.getVars() ] # use list comprehension
df_var = pd.DataFrame(varInfo) # convert to pandas dataframe
df_var.columns=['Variable Name','LB','UB'] # Add column headers
display(df_var)

# Linear constraint info
constrInfo = [(c.constrName, model.getRow(c), c.Sense, c.RHS) for c in model.getConstrs() ]
df_constr = pd.DataFrame(constrInfo)
df_constr.columns=['Constraint Name','Constraint equation', 'Sense','RHS']
display(df_constr)

request_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        temp = pd.DataFrame({"From": [arc[0]], "To": [arc[1]], "Flow": [flow[arc].x]})
        request_flow = pd.concat([request_flow, temp], ignore_index=True)
request_flow
         
# Simple version of model relax
# if model.status == GRB.INFEASIBLE:
#     vars = model.getVars()
#     ubpen = [1.0]*model.numVars
#     model.feasRelax(1, False, vars, None, ubpen, None, None)
#     model.optimize()

# # Simple version of model relax
# if model.status == GRB.INFEASIBLE:
#     model.feasRelaxS(1, False, False, True)
#     model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 19 rows, 14 columns and 40 nonzeros
Model fingerprint: 0xd8aa0827
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 3e+02]
Presolve removed 11 rows and 5 columns
Presolve time: 0.01s
Presolved: 8 rows, 10 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+03   6.875000e+01   0.000000e+00      0s
       4    1.2300000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.230000000e+04


,Variable Name,LB,UB
0,"flow[user_1,A_start_1]",0.0,inf
1,"flow[user_2,A_start_2]",0.0,inf
2,"flow[user_1,A_start_2]",0.0,inf
3,"flow[user_2,A_start_1]",0.0,inf
4,"flow[A_start_1,A_end_1]",0.0,inf
5,"flow[A_start_2,A_end_2]",0.0,inf
6,"flow[A_end_1,B_start_1]",0.0,inf
7,"flow[A_end_2,B_start_2]",0.0,inf
8,"flow[A_end_1,B_start_2]",0.0,inf
9,"flow[A_end_2,B_start_1]",0.0,inf


,Constraint Name,Constraint equation,Sense,RHS
0,source[user_1],"flow[user_1,A_start_1] + flow[user_1,A_start_2]",=,150.0
1,source[user_2],"flow[user_2,A_start_2] + flow[user_2,A_start_1]",=,150.0
2,destination[dst],"flow[B_end_1,dst] + flow[B_end_2,dst]",=,300.0
3,flow_conservation[A_start_1],"-1.0 flow[user_1,A_start_1] + -1.0 flow[user_2...",=,0.0
4,flow_conservation[A_end_1],"-1.0 flow[A_start_1,A_end_1] + flow[A_end_1,B_...",=,0.0
5,flow_conservation[A_start_2],"-1.0 flow[user_2,A_start_2] + -1.0 flow[user_1...",=,0.0
6,flow_conservation[A_end_2],"-1.0 flow[A_start_2,A_end_2] + flow[A_end_2,B_...",=,0.0
7,flow_conservation[B_start_1],"-1.0 flow[A_end_1,B_start_1] + -1.0 flow[A_end...",=,0.0
8,flow_conservation[B_end_1],"-1.0 flow[B_start_1,B_end_1] + flow[B_end_1,dst]",=,0.0
9,flow_conservation[B_start_2],"-1.0 flow[A_end_2,B_start_2] + -1.0 flow[A_end...",=,0.0


,From,To,Flow
0,user_1,A_start_1,150.0
1,user_2,A_start_2,150.0
2,A_start_1,A_end_1,150.0
3,A_start_2,A_end_2,150.0
4,A_end_1,B_start_1,150.0
5,A_end_2,B_start_2,150.0
6,B_start_1,B_end_1,150.0
7,B_start_2,B_end_2,150.0
8,B_end_1,dst,150.0
9,B_end_2,dst,150.0
